If not already done run this in the top level folder:
```
pip install -r requirements.txt
```




In [3]:
import json
import requests
import os
import openai
from enum import Enum
from pydantic import BaseModel
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")
    
API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
API_VERSION = os.getenv("OPENAI_API_VERSION")
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")


client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = os.getenv("AZURE_OPENAI_VERSION")
)
deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
model_name = os.getenv("AZURE_OPENAI_COMPLETION_MODEL")

Found Azure OpenAI API Base Endpoint: https://cog-fkplarx5db7we.openai.azure.com/


To see if stuff works in general, you can run this.

In [ ]:
# response = client.chat.completions.create(
#     model = model_name,    
#     messages = [{"role" : "assistant", "content" : "The one thing I love more than anything else is "}],
# )

# print(response)


This is the object model for receiving questions.

In [4]:

class QuestionType(str, Enum):
    multiple_choice = "multiple_choice"
    true_false = "true_false"
    popular_choice = "popular_choice"
    estimation = "estimation"

class Ask(BaseModel):
    question: str | None = None
    type: QuestionType
    correlationToken: str | None = None

class Answer(BaseModel):
    answer: str
    correlationToken: str | None = None
    promptTokensUsed: int | None = None
    completionTokensUsed: int | None = None


In [5]:
# async def ask_question(ask: Ask):
#     """
#     Ask a question
#     """
#     print('Sending a request to openai')

#     # Send a completion call to generate an answer
#     print('Sending a request to openai --')

#     start_phrase = ask.question
#     if(ask.type == QuestionType.multiple_choice):
#         response = client.chat.completions.create(
#         model = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"),
#         messages = [{"role" : "assistant", "content" : start_phrase}, 
#                     { "role" : "system", "content" : "answer with the character or number of the right option only . For example: Question: Which of the following is  a color? a. Tree b. Flower c. Green  Answer: c "}]
          
#     )
        
#     if(ask.type == QuestionType.true_false):
#         response = client.chat.completions.create(
#         model = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"),
#         messages = [{"role" : "assistant", "content" : start_phrase}, 
#                 { "role" : "system", "content" : "respond with true or false only. "}]
      
#     )

#     print(response.choices[0].message.content)
        
#     answer = Answer(answer=response.choices[0].message.content)
#     answer.correlationToken = ask.correlationToken
#     answer.promptTokensUsed = response.usage.prompt_tokens
#     answer.completionTokensUsed = response.usage.completion_tokens

#     print('Answer:', answer)
#     return answer

YOUR Mission: Adjust the function below and reuse it in the main.py file later to deploy to Azure and to update your service


In [6]:

async def ask_question(ask: Ask):
    """
    Ask a question
    """

    # Send a completion call to generate an answer
    print('Sending a request to openai')
    start_phrase = ask.question
    response = client.chat.completions.create(
        model = deployment_name,
        messages = [{"role" : "assistant", "content" : start_phrase}],
    )

    print(response.choices[0].message.content)
    print(response)
    answer = Answer(answer=response.choices[0].message.content)
    answer.correlationToken = ask.correlationToken
    answer.promptTokensUsed = response.usage.prompt_tokens
    answer.completionTokensUsed = response.usage.completion_tokens

    return answer

Use this snippet to try your method with several questions.

In [7]:

ask = Ask(question="Which of the following is a color? a. Tree b. Flower c. Green", type=QuestionType.multiple_choice)
answer = await ask_question(ask)
print('Answer:', answer)


Sending a request to openai
c. Green
ChatCompletion(id='chatcmpl-9RZul43FC7lg1qVPjwLCU357OEQYY', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='c. Green', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1716359939, model='gpt-35-turbo', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=3, prompt_tokens=24, total_tokens=27), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])
Answer: answer='c. Green' correlationToken=None p

Make sure you transfer your code changes into main.py (or additional files). Then redeploy your container using this command.
```
bash ./azd-hooks/deploy.sh phase1 $AZURE_ENV_NAME
```
